In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch
import xarray as xr
import numpy as np
import pandas as pd
import calendar
import seaborn as sns
import seaborn_image as isns
import calendar

In [2]:
mesh_mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/mesh_mask.nc')

In [3]:
mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/tmaskm.nc')

In [4]:
mask = mask.tmask.isel(time=0)

### Ocean slicing and mixed layer depth average

In [5]:
# calculate volume for each grid cell

dx = mesh_mask.e1t 
dy = mesh_mask.e2t
grid_cell_area = dx * dy # m2

In [6]:
gridded_mesh = grid_cell_area.isel(t=0)

In [7]:
# apply mask to weighted grid cells

ocean_area = gridded_mesh * mask

In [8]:
ocean_area_sliced = ocean_area.sel(x=slice(520,595),y=slice(330,435),z=0)

In [9]:
# alkalinity weighed only over the z axis (depth)

In [10]:
layer_depth = mask.isel(x=slice(520,595),y=slice(330,435),z=slice(0,7))

In [11]:
layer_depth = layer_depth.sum('z') # m

In [12]:
layer_depth = layer_depth.where(layer_depth)

In [13]:
layer_depth = layer_depth.to_dataset(name='tmask')

In [14]:
layer_depth = layer_depth.to_dataframe()

In [15]:
# condition

max_depth = [
    (layer_depth.tmask == 1),
    (layer_depth.tmask == 2),
    (layer_depth.tmask == 3),
    (layer_depth.tmask == 4),
    (layer_depth.tmask == 5),
    (layer_depth.tmask == 6),
    (layer_depth.tmask == 7)
]

values_depth = [3.046773, 9.454049, 16.363966, 23.89871, 32.20929, 41.481853, 51.94513]

layer_depth['tmask'] = np.select(max_depth, values_depth)

In [16]:
layer_depth = layer_depth.drop(columns='time')

In [17]:
layer_depth = layer_depth.to_xarray()

In [18]:
layer_depth = layer_depth.where(layer_depth)

In [19]:
ocean_area_sliced_dp = ocean_area_sliced.isel(x=slice(58,60),y=slice(41,43))

### Alkalinity weighted only over the z axis (depth)

In [20]:
alk_weighted_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_2.6_cropped.nc',
                                         decode_times=True).isel(deptht=slice(0,7))

In [21]:
alk_weighted_baseline = (alk_weighted_baseline.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [22]:
alk_0_base = alk_weighted_baseline.ALK.isel(deptht=0) * alk_weighted_baseline.deptht[0] # mmol/m3 * m
alk_1_base = alk_weighted_baseline.ALK.isel(deptht=1) * (alk_weighted_baseline.deptht[1] - alk_weighted_baseline.deptht[0])
alk_2_base = alk_weighted_baseline.ALK.isel(deptht=2) * (alk_weighted_baseline.deptht[2] - alk_weighted_baseline.deptht[1])
alk_3_base = alk_weighted_baseline.ALK.isel(deptht=3) * (alk_weighted_baseline.deptht[3] - alk_weighted_baseline.deptht[2])
alk_4_base = alk_weighted_baseline.ALK.isel(deptht=4) * (alk_weighted_baseline.deptht[4] - alk_weighted_baseline.deptht[3])
alk_5_base = alk_weighted_baseline.ALK.isel(deptht=5) * (alk_weighted_baseline.deptht[5] - alk_weighted_baseline.deptht[4])
alk_6_base = alk_weighted_baseline.ALK.isel(deptht=6) * (alk_weighted_baseline.deptht[6] - alk_weighted_baseline.deptht[5])

In [23]:
weighted_base_alk = (alk_0_base + alk_1_base + alk_2_base + alk_3_base + alk_4_base + alk_5_base + alk_6_base) / layer_depth.tmask # mmol/m3 * m / m

In [24]:
weighted_base_alk = weighted_base_alk.to_dataset(name='ALK')

In [25]:
alk_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_OAE_2.6_cropped.nc',
                                    decode_times=True).isel(deptht=slice(0, 7))

In [26]:
alk_weighted_oae = (alk_weighted_oae.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [27]:
alk_0_oae = alk_weighted_oae.ALK.isel(deptht=0) * alk_weighted_oae.deptht[0] # mmol/m3 * m
alk_1_oae = alk_weighted_oae.ALK.isel(deptht=1) * (alk_weighted_oae.deptht[1] - alk_weighted_oae.deptht[0])
alk_2_oae = alk_weighted_oae.ALK.isel(deptht=2) * (alk_weighted_oae.deptht[2] - alk_weighted_oae.deptht[1])
alk_3_oae = alk_weighted_oae.ALK.isel(deptht=3) * (alk_weighted_oae.deptht[3] - alk_weighted_oae.deptht[2])
alk_4_oae = alk_weighted_oae.ALK.isel(deptht=4) * (alk_weighted_oae.deptht[4] - alk_weighted_oae.deptht[3])
alk_5_oae = alk_weighted_oae.ALK.isel(deptht=5) * (alk_weighted_oae.deptht[5] - alk_weighted_oae.deptht[4])
alk_6_oae = alk_weighted_oae.ALK.isel(deptht=6) * (alk_weighted_oae.deptht[6] - alk_weighted_oae.deptht[5])

In [28]:
alk_weighted_oae = (alk_0_oae + alk_1_oae + alk_2_oae + alk_3_oae + alk_4_oae + alk_5_oae + alk_6_oae) / layer_depth.tmask # mmol/m3 * m / m

In [29]:
alk_weighted_oae = alk_weighted_oae.to_dataset(name='ALK')

In [30]:
#alk_weighted_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_amplitude_base.nc')
#alk_weighted_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_amplitude_oae.nc')

In [31]:
alk_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_OAE_2.6_cropped.nc',
                                    decode_times=True).isel(deptht=slice(0, 7))

In [32]:
alk_weighted_oae = (alk_weighted_oae.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [33]:
alk_0_oae = alk_weighted_oae.ALK.isel(deptht=0) * alk_weighted_oae.deptht[0] # mmol/m3 * m
alk_1_oae = alk_weighted_oae.ALK.isel(deptht=1) * (alk_weighted_oae.deptht[1] - alk_weighted_oae.deptht[0])
alk_2_oae = alk_weighted_oae.ALK.isel(deptht=2) * (alk_weighted_oae.deptht[2] - alk_weighted_oae.deptht[1])
alk_3_oae = alk_weighted_oae.ALK.isel(deptht=3) * (alk_weighted_oae.deptht[3] - alk_weighted_oae.deptht[2])
alk_4_oae = alk_weighted_oae.ALK.isel(deptht=4) * (alk_weighted_oae.deptht[4] - alk_weighted_oae.deptht[3])
alk_5_oae = alk_weighted_oae.ALK.isel(deptht=5) * (alk_weighted_oae.deptht[5] - alk_weighted_oae.deptht[4])
alk_6_oae = alk_weighted_oae.ALK.isel(deptht=6) * (alk_weighted_oae.deptht[6] - alk_weighted_oae.deptht[5])

In [34]:
alk_weighted_oae = (alk_0_oae + alk_1_oae + alk_2_oae + alk_3_oae + alk_4_oae + alk_5_oae + alk_6_oae) / layer_depth.tmask # mmol/m3 * m / m

In [35]:
alk_weighted_oae = alk_weighted_oae.to_dataset(name='ALK')

In [36]:
#alk_weighted_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_amplitude_base.nc')
#alk_weighted_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_amplitude_oae.nc')

### DIC weighted only over the z axis (depth)

In [37]:
dic_weighted_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_Baseline_2.6_cropped.nc', decode_times=True)

In [38]:
dic_weighted_baseline = (dic_weighted_baseline.DIC * 1.025).to_dataset(name='DIC') # mmol/m3

In [39]:
dic_0_base = dic_weighted_baseline.DIC.isel(deptht=0) * dic_weighted_baseline.deptht[0] # mmol/m3 * m
dic_1_base = dic_weighted_baseline.DIC.isel(deptht=1) * (dic_weighted_baseline.deptht[1] - dic_weighted_baseline.deptht[0])
dic_2_base = dic_weighted_baseline.DIC.isel(deptht=2) * (dic_weighted_baseline.deptht[2] - dic_weighted_baseline.deptht[1])
dic_3_base = dic_weighted_baseline.DIC.isel(deptht=3) * (dic_weighted_baseline.deptht[3] - dic_weighted_baseline.deptht[2])
dic_4_base = dic_weighted_baseline.DIC.isel(deptht=4) * (dic_weighted_baseline.deptht[4] - dic_weighted_baseline.deptht[3])
dic_5_base = dic_weighted_baseline.DIC.isel(deptht=5) * (dic_weighted_baseline.deptht[5] - dic_weighted_baseline.deptht[4])
dic_6_base = dic_weighted_baseline.DIC.isel(deptht=6) * (dic_weighted_baseline.deptht[6] - dic_weighted_baseline.deptht[5])

In [40]:
dic_weighted_base = (dic_0_base + dic_1_base + dic_2_base + dic_3_base + dic_4_base + dic_5_base + dic_6_base) / layer_depth.tmask

In [41]:
dic_weighted_base = dic_weighted_base.to_dataset(name='DIC')

In [42]:
dic_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_OAE_2.6_cropped.nc', decode_times=True)

In [43]:
dic_weighted_oae = (dic_weighted_oae.DIC * 1.025).to_dataset(name='DIC') # mmol/m3

In [44]:
dic_0_oae = dic_weighted_oae.DIC.isel(deptht=0) * dic_weighted_oae.deptht[0] # mmol/m3 * m
dic_1_oae = dic_weighted_oae.DIC.isel(deptht=1) * (dic_weighted_oae.deptht[1] - dic_weighted_oae.deptht[0]) 
dic_2_oae = dic_weighted_oae.DIC.isel(deptht=2) * (dic_weighted_oae.deptht[2] - dic_weighted_oae.deptht[1])
dic_3_oae = dic_weighted_oae.DIC.isel(deptht=3) * (dic_weighted_oae.deptht[3] - dic_weighted_oae.deptht[2])
dic_4_oae = dic_weighted_oae.DIC.isel(deptht=4) * (dic_weighted_oae.deptht[4] - dic_weighted_oae.deptht[3])
dic_5_oae = dic_weighted_oae.DIC.isel(deptht=5) * (dic_weighted_oae.deptht[5] - dic_weighted_oae.deptht[4])
dic_6_oae = dic_weighted_oae.DIC.isel(deptht=6) * (dic_weighted_oae.deptht[6] - dic_weighted_oae.deptht[5])

In [45]:
dic_weighted_oae = (dic_0_oae + dic_1_oae + dic_2_oae + dic_3_oae + dic_4_oae + dic_5_oae + dic_6_oae) / layer_depth.tmask # mmol/m3 * m / m

In [46]:
dic_weighted_oae = dic_weighted_oae.to_dataset(name='DIC')

In [47]:
#dic_weighted_base.to_netcdf('/Users/chiaraciscato/Desktop/geomar/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_amplitude_base.nc')
#dic_weighted_oae.to_netcdf('/Users/chiaraciscato/Desktop/geomar/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_amplitude_oae.nc')

### Alkalinity weighted over x,y,z

In [48]:
sliced_alk_baseline = ocean_area_sliced * weighted_base_alk.ALK # baseline

In [49]:
regridded_alk_base = sliced_alk_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [50]:
regridded_alk_base = regridded_alk_base.to_dataset(name='ALK')

In [51]:
sliced_alk_oae = ocean_area_sliced * alk_weighted_oae.ALK # oae

In [52]:
regridded_alk_oae = sliced_alk_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [53]:
regridded_alk_oae = regridded_alk_oae.to_dataset(name='ALK')

In [54]:
#regridded_alk_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_2.6_weighted.nc')
#regridded_alk_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_oae_2.6_weighted.nc')

### Alkalinity surface weighted over x and y

In [55]:
alk_surface_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_2.6_cropped.nc',
                                         decode_times=True).isel(deptht=slice(0,1))

In [56]:
alk_surface_baseline = (alk_surface_baseline.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [57]:
alk_surface_baseline = ocean_area_sliced * alk_surface_baseline.ALK # mmol/m3 * m2

In [58]:
regridded_alk_surf_base = alk_surface_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) # mmol/m3 * m2 / m2

In [59]:
regridded_alk_surf_base = regridded_alk_surf_base.to_dataset(name='ALK')

In [60]:
alk_surface_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_OAE_2.6_cropped.nc',
                                         decode_times=True).isel(deptht=slice(0,1))

In [61]:
alk_surface_oae = (alk_surface_oae.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [62]:
alk_surface_oae = ocean_area_sliced * alk_surface_oae.ALK # mmol/m3 * m2

In [63]:
regridded_alk_surf_oae = alk_surface_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) # mmol/m3 * m2 / m2

In [64]:
regridded_alk_surf_oae = regridded_alk_surf_oae.to_dataset(name='ALK')

In [65]:
#regridded_alk_surf_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_surf_2.6_weighted.nc')
#regridded_alk_surf_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_oae_surf_2.6_weighted.nc')

### Alkalinity weighted over x and y for the first 150 metres:

In [160]:
alk_col_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_2.6_cropped.nc', decode_times=True)

In [161]:
alk_col_baseline = (alk_col_baseline.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [162]:
alk_col_baseline = ocean_area_sliced * alk_col_baseline.ALK # mmol/m3 * m2

In [163]:
regridded_alk_col_base = alk_col_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) # mmol/m3 * m2 / m2

In [164]:
regridded_alk_col_base = regridded_alk_col_base.to_dataset(name='ALK')

In [165]:
alk_col_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_OAE_2.6_cropped.nc', decode_times=True)

In [166]:
alk_col_oae = (alk_col_oae.ALK * 1.025).to_dataset(name='ALK') # mmol/m3

In [167]:
alk_col_oae = ocean_area_sliced * alk_col_oae.ALK # mmol/m3 * m2

In [168]:
regridded_alk_col_oae = alk_col_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) # mmol/m3 * m2 / m2

In [169]:
regridded_alk_col_oae = regridded_alk_col_oae.to_dataset(name='ALK')

In [76]:
#regridded_alk_col_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_col_2.6_weighted.nc')
#regridded_alk_col_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_oae_col_2.6_weighted.nc')

### DIC weighted over x,y,z

In [77]:
sliced_dic_baseline = ocean_area_sliced * dic_weighted_base.DIC # mmol/m3 * m2

In [78]:
regridded_dic_base = sliced_dic_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) # mmol/m3 * m2 / m2

In [79]:
regridded_dic_base = regridded_dic_base.to_dataset(name='DIC')

In [80]:
sliced_dic_oae = ocean_area_sliced * dic_weighted_oae.DIC # oae

In [81]:
regridded_dic_oae = sliced_alk_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [82]:
regridded_dic_oae = regridded_dic_oae.to_dataset(name='DIC')

In [83]:
#regridded_dic_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_baseline_2.6_weighted.nc')
#regridded_dic_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_oae_2.6_weighted.nc')

### Weighted alkalinity at location point

In [84]:
alk_weighted_baseline_dp = weighted_base_alk.isel(x=slice(58,60),y=slice(41,43))

In [85]:
sliced_alk_baseline_dp = ocean_area_sliced_dp * alk_weighted_baseline_dp.ALK

In [86]:
regridded_alk_base_dp = sliced_alk_baseline_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [87]:
regridded_alk_base_dp = regridded_alk_base_dp.to_dataset(name='ALK')

In [88]:
alk_weighted_oae_dp = alk_weighted_oae.isel(x=slice(58,60),y=slice(41,43)) 

In [89]:
sliced_alk_oae_dp = ocean_area_sliced_dp * alk_weighted_oae_dp.ALK

In [90]:
regridded_alk_oae_dp = sliced_alk_oae_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [91]:
regridded_alk_oae_dp = regridded_alk_oae_dp.to_dataset(name='ALK')

In [92]:
#regridded_alk_base_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_baseline_2.6_weighted_dp.nc')
#regridded_alk_oae_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/ALK_oae_2.6_weighted_dp.nc')

### Weighted DIC at location point

In [93]:
dic_weighted_baseline_dp = dic_weighted_base.isel(x=slice(58,60),y=slice(41,43))

In [94]:
sliced_dic_baseline_dp = ocean_area_sliced_dp * dic_weighted_baseline_dp.DIC

In [95]:
regridded_dic_base_dp = sliced_dic_baseline_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [96]:
regridded_dic_base_dp = regridded_dic_base_dp.to_dataset(name='DIC')

In [97]:
dic_weighted_oae_dp = dic_weighted_oae.isel(x=slice(58,60),y=slice(41,43)) 

In [98]:
sliced_dic_oae_dp = ocean_area_sliced_dp * dic_weighted_oae_dp.DIC

In [99]:
regridded_dic_oae_dp = sliced_dic_oae_dp.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])

In [100]:
regridded_dic_oae_dp = regridded_dic_oae_dp.to_dataset(name='DIC')

In [101]:
#regridded_dic_base_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_baseline_2.6_weighted_dp.nc')
#regridded_dic_oae_dp.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/DIC/DIC_oae_2.6_weighted_dp.nc')

### Weighted CO2 flux

In [102]:
co2flux_weighted_base = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/CO2flux/CO2flux_Baseline_2.6_cropped.nc')

In [103]:
sliced_co2flux_baseline = ocean_area_sliced * co2flux_weighted_base.co2flux # baseline

In [104]:
regridded_co2flux_base = sliced_co2flux_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y']) 

In [105]:
regridded_co2flux_base = regridded_co2flux_base.to_dataset(name='co2flux')

In [106]:
co2flux_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/CO2flux/CO2flux_OAE_2.6_cropped.nc')

In [107]:
sliced_co2flux_oae = ocean_area_sliced * co2flux_weighted_oae.co2flux # oae

In [108]:
regridded_co2flux_oae = sliced_co2flux_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [109]:
regridded_co2flux_oae = regridded_co2flux_oae.to_dataset(name='co2flux')

In [110]:
#regridded_co2flux_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/CO2flux/co2flux_baseline_2.6_weighted.nc')
#regridded_co2flux_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/CO2flux/co2flux_oae_2.6_weighted.nc')

### Weighted fCO2

In [111]:
fco2_weighted_base = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/FCO2/fco2_baseline_2.6_cropped.nc')

In [112]:
sliced_fco2_baseline = ocean_area_sliced * fco2_weighted_base.fco2 # baseline

In [113]:
regridded_fco2_base = sliced_fco2_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [114]:
regridded_fco2_base = regridded_fco2_base.to_dataset(name='fco2')

In [115]:
fco2_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/FCO2/fco2_OAE_2.6_cropped.nc')

In [116]:
sliced_fco2_oae = ocean_area_sliced * fco2_weighted_oae.fco2 # oae

In [117]:
regridded_fco2_oae = sliced_fco2_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [118]:
regridded_fco2_oae = regridded_fco2_oae.to_dataset(name='fco2')

In [119]:
#regridded_fco2_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/FCO2/fco2_baseline_2.6_weighted.nc')
#regridded_fco2_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/FCO2/fco2_oae_2.6_weighted.nc')

### Weighted pH

In [120]:
ph_weighted_base = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/pH/ph_baseline_2.6_cropped.nc')

In [121]:
sliced_ph_baseline = ocean_area_sliced * ph_weighted_base.ph # baseline

In [122]:
regridded_ph_base = sliced_ph_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [123]:
regridded_ph_base = regridded_ph_base.to_dataset(name='ph')

In [124]:
ph_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/pH/ph_OAE_2.6_cropped.nc')

In [125]:
sliced_ph_oae = ocean_area_sliced * ph_weighted_oae.ph # oae

In [126]:
regridded_ph_oae = sliced_ph_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [127]:
regridded_ph_oae = regridded_ph_oae.to_dataset(name='ph')

In [128]:
#regridded_ph_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/pH/ph_baseline_2.6_weighted.nc')
#regridded_ph_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/pH/ph_oae_2.6_weighted.nc')

### Weighted SST

In [129]:
sst_weighted_base = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_baseline_2.6_cropped.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_baseline_2.6_cropped.nc'

In [ ]:
sliced_sst_baseline = ocean_area_sliced * sst_weighted_base.sst # baseline

In [ ]:
regridded_sst_base = sliced_sst_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [ ]:
regridded_sst_base = regridded_sst_base.to_dataset(name='sst')

In [ ]:
sst_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_OAE_2.6_cropped.nc')

In [ ]:
sliced_sst_oae = ocean_area_sliced * sst_weighted_oae.sst # oae

In [ ]:
regridded_sst_oae = sliced_sst_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [ ]:
regridded_sst_oae = regridded_sst_oae.to_dataset(name='sst')

In [ ]:
#regridded_sst_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_baseline_2.6_weighted.nc')
#regridded_sst_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/sst_oae_2.6_weighted.nc')

### Weighted Mixed Layer Depth 

In [ ]:
mld_weighted_base = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SWT&MLD/MLD_2.6/MLD_baseline_2.6_cropped.nc')

In [ ]:
sliced_mld_baseline = ocean_area_sliced * mld_weighted_base.mld # baseline

In [ ]:
regridded_mld_base = sliced_mld_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [ ]:
regridded_mld_base = regridded_mld_base.to_dataset(name='mld')

In [ ]:
mld_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/SWT&MLD/MLD_2.6/MLD_OAE_2.6_cropped.nc')

In [ ]:
sliced_mld_oae = ocean_area_sliced * mld_weighted_oae.mld # oae

In [ ]:
regridded_mld_oae = sliced_mld_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [ ]:
regridded_mld_oae = regridded_mld_oae.to_dataset(name='mld')

In [ ]:
#regridded_mld_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/mld_baseline_2.6_weighted.nc')
#regridded_mld_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/mld_oae_2.6_weighted.nc')